# Aggregations and JOINs
Apache Spark&trade; and Databricks&reg; allow you to create on-the-fly data lakes.

### Getting Started

Run the following cell to configure our "classroom."

In [3]:
%run "./Includes/Classroom-Setup"

-sandbox
## Basic Aggregations

Using <a "https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions" target="_blank">built-in Spark functions</a>, you can aggregate data in various ways. 

Run the cell below to compute the average of all salaries in the people DataFrame.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> By default, you get a floating point value.

In [5]:
peopleDF = spark.read.parquet("/mnt/training/dataframes/people-10m.parquet")

In [6]:
from pyspark.sql.functions import avg
avgSalaryDF = peopleDF.select(avg("salary").alias("averageSalary"))

avgSalaryDF.show()

Convert that value to an integer using the `round()` function. See
<a href "https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$" class="text-info">the documentation for <tt>round()</tt></a>
for more details.

In [8]:
from pyspark.sql.functions import round
roundedAvgSalaryDF = avgSalaryDF.select(round("averageSalary").alias("roundedAverageSalary"))

roundedAvgSalaryDF.show()

In addition to the average salary, what are the maximum and minimum salaries?

In [10]:
from pyspark.sql.functions import min, max
salaryDF = peopleDF.select(max("salary").alias("max"), min("salary").alias("min"), round(avg("salary")).alias("averageSalary"))

salaryDF.show()

## Joining Two Data Sets

Correlate the data in two data sets using a DataFrame join. 

The `people` data set has 10 million names in it. 

> How many of the first names appear in Social Security data files? 

To find out, use the Social Security data set with first name popularity data from the United States Social Security Administration. 

For every year from 1880 to 2014, `dbfs:/mnt/training/ssn/names-1880-2016.parquet/` lists the first names of people born in that year, their gender, and the total number of people given that name. 

By joining the `people` data set with `names-1880-2016`, weed out the names that aren't represented in the Social Security data.

(In a real application, you might use a join like this to filter out bad data.)

Start by taking a look at what the social security data set looks like. Each year is its own directory.

In [13]:
%fs ls dbfs:/mnt/training/ssn/names-1880-2016.parquet/

Let's load this file into a DataFrame and look at the data.

In [15]:
ssaDF = spark.read.parquet("/mnt/training/ssn/names-1880-2016.parquet/")

display(ssaDF)

Next, with a quick count of distinct names, get an idea of how many distinct names there are in each of the tables.

DataFrames have a `distinct` method just for this purpose.

In [17]:
peopleDistinctNamesDF = peopleDF.select("firstName").distinct()

In [18]:
peopleDistinctNamesDF.count()

-sandbox
In preparation for the join, let's rename the `firstName` column to `ssaFirstName` in the Social Security DataFrame.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Question to ponder: why would we want to do this?

In [20]:
ssaDistinctNamesDF = ssaDF.select("firstName").withColumnRenamed("firstName",'ssaFirstName').distinct()

Count how many distinct names in the Social Security DataFrame.

In [22]:
ssaDistinctNamesDF.count()

Now join the two DataFrames.

In [24]:
from pyspark.sql.functions import col
joinedDF = peopleDistinctNamesDF.join(ssaDistinctNamesDF, col("firstName") == col("ssaFirstName"))

How many are there?

In [26]:
joinedDF.count()

-sandbox
## Exercise 1

In the tables above, some of the salaries in the `peopleDF` DataFrame are negative. 

These salaries represent bad data. 

Your job is to convert all the negative salaries to positive ones, and then sort the top 20 people by their salary.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** See the Apache Spark documentation, <a href="https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$" target="_blank">built-in functions</a>.

### Step 1
Create a DataFrame`PeopleWithFixedSalariesDF`, where all the negative salaries have been converted to positive numbers.

In [29]:
# ANSWER
from pyspark.sql.functions import abs
peopleWithFixedSalariesDF = peopleDF.select("firstName","middleName","lastName","gender","birthDate","ssn",abs(col("salary")).alias('salary'))
display(peopleWithFixedSalariesDF)

In [30]:
# TEST - Run this cell to test your solution.

belowZero = peopleWithFixedSalariesDF.filter(peopleWithFixedSalariesDF["salary"] < 0).count()
dbTest("DF-L3-belowZero", 0, belowZero)

print("Tests passed!")

### Step 2

Starting with the `peopleWithFixedSalariesDF` DataFrame, create another DataFrame called `PeopleWithFixedSalariesSortedDF` where:
0. The data set has been reduced to the first 20 records.
0. The records are sorted by the column `salary` in ascending order.

In [32]:
# ANSWER
peopleWithFixedSalariesSortedDF = peopleWithFixedSalariesDF.select("*").orderBy("salary").limit(20)

In [33]:
# TEST - Run this cell to test your solution.

resultsDF = peopleWithFixedSalariesSortedDF.select("salary")
dbTest("DF-L3-count", 20, resultsDF.count())

print("Tests passed!")

In [34]:
# TEST - Run this cell to test your solution.

from pyspark.sql import Row

results = resultsDF.collect()

dbTest("DF-L3-fixedSalaries-0", Row(salary=2), results[0])
dbTest("DF-L3-fixedSalaries-1", Row(salary=3), results[1])
dbTest("DF-L3-fixedSalaries-2", Row(salary=4), results[2])

dbTest("DF-L3-fixedSalaries-10", Row(salary=19), results[10])
dbTest("DF-L3-fixedSalaries-11", Row(salary=19), results[11])
dbTest("DF-L3-fixedSalaries-12", Row(salary=20), results[12])

dbTest("DF-L3-fixedSalaries-17", Row(salary=28), results[17])
dbTest("DF-L3-fixedSalaries-18", Row(salary=30), results[18]) 
dbTest("DF-L3-fixedSalaries-19", Row(salary=31), results[19]) 

print("Tests passed!")

## Exercise 2

As a refinement, assume all salaries under $20,000 represent bad rows and filter them out.

Additionally, categorize each person's salary into $10K groups.

### Step 1
 Starting with the `peopleWithFixedSalariesDF` DataFrame, create a DataFrame called `peopleWithFixedSalaries20KDF` where:
0. The data set excludes all records where salaries are below $20K.
0. The data set includes a new column called `salary10k`, that should be the salary in groups of 10,000. For example:
  * A salary of 23,000 should report a value of "2".
  * A salary of 57,400 should report a value of "6".
  * A salary of 1,231,375 should report a value of "123".

In [37]:
# ANSWER

peopleWithFixedSalaries20KDF = peopleWithFixedSalariesDF.select("*").withColumn("salary10k", round(col("salary")/10000)).filter("salary >= 20000")

In [38]:
# TEST - Run this cell to test your solution.

below20K = peopleWithFixedSalaries20KDF.filter("salary < 20000").count()
 
dbTest("DF-L3-count-salaries", 0, below20K)  

print("Tests passed!")

In [39]:
# TEST - Run this cell to test your solution.

from pyspark.sql.functions import count
results = (peopleWithFixedSalaries20KDF 
  .select("salary10k") 
  .groupBy("salary10k") 
  .agg(count("*").alias("total")) 
  .orderBy("salary10k") 
  .limit(5) 
  .collect()
)

dbTest("DF-L3-countSalaries-0", Row(salary10k=2.0, total=43792), results[0])
dbTest("DF-L3-countSalaries-1", Row(salary10k=3.0, total=212630), results[1])
dbTest("DF-L3-countSalaries-2", Row(salary10k=4.0, total=536536), results[2])
dbTest("DF-L3-countSalaries-3", Row(salary10k=5.0, total=1055261), results[3])
dbTest("DF-L3-countSalaries-4", Row(salary10k=6.0, total=1623248), results[4])

print("Tests passed!")

## Exercise 3

Using the `peopleDF` DataFrame, count the number of females named Caren who were born before March 1980.

### Step 1

Starting with `peopleDF`, create a DataFrame called `carensDF` where:
0. The result set has a single record.
0. The data set has a single column named `total`.
0. The result counts only 
  * Females (`gender`)
  * First Name is "Caren" (`firstName`)
  * Born before March 1980 (`birthDate`)

In [42]:
# ANSWER
carensDF = (peopleDF 
  .filter("birthDate < '1980-03-01' ") 
  .filter("firstName = 'Caren' ") 
  .filter("gender = 'F' ") 
  .agg(count("*").alias("total"))
)

In [43]:
# TEST - Run this cell to test your solution.

rows = carensDF.collect()

dbTest("DF-L3-carens-len", 1, len(rows))
dbTest("DF-L3-carens-total", Row(total=750), rows[0])

print("Tests passed!")

## Review Questions
**Q:** What is the DataFrame equivalent of the SQL statement `SELECT count(*) AS total`  
**A:** ```.agg(count("*").alias("total"))```

**Q:** What is the DataFrame equivalent of the SQL statement 
```SELECT firstName FROM PeopleDistinctNames INNER JOIN SSADistinctNames ON firstName = ssaFirstName```  
**A:** 
`peopleDistinctNamesDF.join(ssaDistinctNamesDF, peopleDistinctNamesDF(col("firstName")) == col("ssaFirstName"))`

## Next Steps

* Do the [Challenge Exercise]($./Optional/03-Joins-Aggregations).
* Start the next lesson, [Accessing Data]($./04-Accessing-Data).

## Additional Topics & Resources

* <a href="http://spark.apache.org/docs/latest/sql-programming-guide.html" target="_blank">Spark SQL, DataFrames and Datasets Guide</a>
* <a href="https://databricks.com/blog/2017/08/31/cost-based-optimizer-in-apache-spark-2-2.html" target="_blank">Cost-based Optimizer in Apache Spark 2.2</a>